<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.7 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

package not found, installing w/ pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.2 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=d962a288af12e392

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135743 sha256=4ee384479f36a7b4e8c95ab37ef3f9bf0689394a2cee26c7e2c6f0d2e8bb625b
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236757 sha256=d9c00d2f8cb1cd77c5d8b2a4b4137c1590b026a0ab149d9b7d5606495fb33989
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=DtxFqA27snwMvXpuds0lAMLGo-PtTOhpWFRwzdtMibo&tc=7azH9jMG0fEAJOmIJ0CxyThXAUbReRHgotEBWalxXcg&cc=Wu19HygrsAzMYjdplOtBFeQj-HHQ5KbCB9x5Wmr4wKU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhV4q6_xaz7MxGIrR1WecZLuqS12_U6rBYn_l9oInigMA8cNv59MeZw

Successfully saved authorization token.


In [7]:
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')


# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))
mines.sample(3)

Data dimensions: (25124, 31)


,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
13861,13862,ADDINGTON ENTERPRISES INC,ADDINGTON ENTERPRISES INC,7,PUMPKING CENTER,8450041,15355,NaN,KY8450041,PRINCESS #3,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,11095.872521,6.733995e+05,"MULTIPOLYGON (((-9219680.334 4646775.067, -921..."
2753,2754,"HAROLD KEENE COAL COMPANY, INC.","HAROLD KEENE COAL COMPANY, INC.",1,THE BANNER MINE,1601982,NaN,NaN,VA1601982,"UPPER BANNER, BIG FORK",...,0.0,0.0,NaN,1.0,NaN,1,https://www.energy.virginia.gov/,14103.996287,9.764467e+05,"MULTIPOLYGON (((-9125559.534 4452656.438, -912..."
23115,23116,"ERP ENVIRONMENTAL FUND, INC.",NaN,1,WESTRIDGE SOUTH NO. 1 SURFACE,S500404,NaN,NaN,WVS500404,NaN,...,0.0,0.0,NaN,0.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,17682.574983,2.260135e+06,"MULTIPOLYGON (((-9125069.377 4587111.829, -912..."


In [8]:
mines.sample(10)

,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
12540,12541,PROGRESS LAND CORPORATION,PROGRESS LAND CORPORATION,7,NaN,8365320,10427,NaN,KY8365320,#2 ELKHORN,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,375.317890,9.338828e+03,"MULTIPOLYGON (((-9205416.626 4484273.996, -920..."
5518,5519,NaN,,4,NaN,0450008,NaN,NaN,KY0450008,,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,3753.678433,1.320977e+05,"MULTIPOLYGON (((-9214725.536 4644380.172, -921..."
3334,3335,REVELATION ENERGY LLC,REVELATION ENERGY LLC,7,NaN,8980928,18193,NaN,KY8980928,"LITTLE FIRECLAY, FIRECLAY, HAMLIN, TAYLOR",...,0.0,1.0,0.0,1.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,26474.105487,3.898104e+06,"MULTIPOLYGON (((-9153872.920 4517230.278, -915..."
5881,5882,NaN,,4,NaN,0585020,NaN,NaN,KY0585020,,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,3042.758969,4.090944e+04,"MULTIPOLYGON (((-9202102.001 4544525.219, -920..."
12027,12028,NaN,NaN,4,NaN,408075X,NaN,NaN,KY408075X,NaN,...,NaN,NaN,NaN,NaN,PRE-LAW,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,5652.331426,3.102596e+05,"MULTIPOLYGON (((-9242295.290 4610054.698, -924..."
14447,14448,NEALY COAL COMPANY INC,NEALY COAL COMPANY INC,7,NaN,8600238,NaN,NaN,KY8600238,ELKHORN #3,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,6802.109035,1.716911e+05,"MULTIPOLYGON (((-9217737.850 4484983.816, -921..."
8591,8592,W L COAL CO,W L COAL CO,5,NaN,8630085,NaN,NaN,KY8630085,UNNAMED,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,1486.531070,2.157105e+04,"MULTIPOLYGON (((-9365443.485 4443484.552, -936..."
10303,10304,NaN,NaN,4,NaN,528976X,NaN,NaN,KY528976X,NaN,...,NaN,NaN,NaN,NaN,PRE-LAW,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,2767.257779,9.893678e+04,"MULTIPOLYGON (((-9332593.480 4405659.185, -933..."
17455,17456,VIKING ACQUISITION GROUP LLC,VIKING ACQUISITION GROUP LLC,1,NaN,8980873,NaN,NaN,KY8980873,"WHITESBURG, HAZARD 4, TAYLOR",...,0.0,0.0,0.0,1.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,20719.404603,7.655965e+05,"MULTIPOLYGON (((-9202404.727 4481288.871, -920..."
21953,21954,"WV RECLAIM CO, LLC","RELGIS, INC",1,LICK BR. NO. 2,U028800,NaN,NaN,WVU028800,NaN,...,NaN,NaN,NaN,NaN,,2,https://dep.wv.gov/dmr/Pages/default.aspx,7074.445602,2.475409e+05,"MULTIPOLYGON (((-9033602.746 4610512.168, -903..."


In [9]:

mines = mines.drop(columns=['permittee'])
mines = mines.drop(columns=['mine_name'])
mines = mines.drop(columns=['msha_id'])
mines = mines.drop(columns=['state_tribe_mine_id'])
mines = mines.drop(columns=['mountaintop'])
mines = mines.drop(columns=['steep_slope'])
mines = mines.drop(columns=['highwall'])
mines = mines.drop(columns=['auger'])
mines = mines.drop(columns=['comment'])
mines = mines.rename(columns={'OBJECTID': 'shapeName'})

In [10]:
mines = mines.drop(columns=['coal_bed_names'])
mines = mines.drop(columns=['date_of_map'])
mines = mines.drop(columns=['inspectable_unit_status'])
mines = mines.drop(columns=['permit_approval_date'])
mines = mines.drop(columns=['edit_date'])
mines = mines.drop(columns=['area_mine'])
mines = mines.drop(columns=['contour'])

In [11]:
mines = mines.drop(columns=['permit_application_type'])
mines = mines.drop(columns=['reported_area'])

In [12]:
mines = mines.drop(columns=['permit_application_date'])

In [13]:
mines

,shapeName,company,coalmine_op_status,permit_id,national_id,post_smcra,calculated_area,contact,information_link,shape_Length,shape_Area,geometry
0,1,HIGGINS COAL CO INC,7,2404,TN2404,1.0,2.220938,4,https://www.osmre.gov/,1220.769913,1.354382e+04,"MULTIPOLYGON (((-9515951.177 4212750.257, -951..."
1,2,MIDNIGHT COAL CO,7,2405,TN2405,1.0,2.752046,4,https://www.osmre.gov/,860.404674,1.678249e+04,"MULTIPOLYGON (((-9516024.476 4212708.540, -951..."
2,3,META ELKHORN MINING CO INC,7,2756,TN2756,1.0,270.299198,4,https://www.osmre.gov/,7219.650616,1.689153e+06,"MULTIPOLYGON (((-9459511.615 4347543.957, -945..."
3,4,JORDAN BROTHERS COAL CO INC,7,2497,TN2497,1.0,3.507990,4,https://www.osmre.gov/,688.042643,2.188255e+04,"MULTIPOLYGON (((-9372128.011 4337061.595, -937..."
4,5,DOUBLE Q INC,7,2284023,TN2284023,1.0,13.751147,4,https://www.osmre.gov/,7994.318732,8.636343e+04,"MULTIPOLYGON (((-9344712.695 4372890.512, -934..."
...,...,...,...,...,...,...,...,...,...,...,...,...
25119,25120,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,154.872159,8,https://surface-mining.alabama.gov/,5370.984081,9.330482e+05,"MULTIPOLYGON (((-9540804.196 4143151.210, -954..."
25120,25121,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,2.410359,8,https://surface-mining.alabama.gov/,534.608228,1.452293e+04,"MULTIPOLYGON (((-9541424.447 4142622.814, -954..."
25121,25122,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,2.430763,8,https://surface-mining.alabama.gov/,507.253927,1.464629e+04,"MULTIPOLYGON (((-9540236.664 4142842.258, -954..."
25122,25123,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,116.325330,8,https://surface-mining.alabama.gov/,4223.617167,7.008642e+05,"MULTIPOLYGON (((-9540725.233 4143212.597, -954..."


In [14]:
# Get the shape geometry for Tennesssee mines
region = eec.gdfToFc(mines)

In [16]:
df = pd.DataFrame()

In [17]:
mines_exploded=mines.explode()
mines_exploded

shapeName                      company  coalmine_op_status permit_id  \
0     0          1          HIGGINS COAL CO INC                   7      2404   
1     0          2             MIDNIGHT COAL CO                   7      2405   
2     0          3   META ELKHORN MINING CO INC                   7      2756   
3     0          4  JORDAN BROTHERS COAL CO INC                   7      2497   
      1          4  JORDAN BROTHERS COAL CO INC                   7      2497   
...            ...                          ...                 ...       ...   
25119 0      25120  Justice Coal Of Alabama LLC                   1     P3946   
25120 0      25121  Justice Coal Of Alabama LLC                   1     P3946   
25121 0      25122  Justice Coal Of Alabama LLC                   1     P3946   
25122 0      25123  Justice Coal Of Alabama LLC                   1     P3946   
25123 0      25124  Justice Coal Of Alabama LLC                   1     P3946   

        national_id  post_smcra  calculated_area  contact  \
0     0      TN2404         1.0         2.220938        4   
1     0      TN2405         1.0         2.752046        4   
2     0      TN2756         1.0       270.299198        4   
3     0      TN2497         1.0         3.507990        4   
      1      TN2497         1.0         3.507990        4   
...             ...         ...              ...      ...   
25119 0     ALP3946         1.0       154.872159        8   
25120 0     ALP3946         1.0         2.410359        8   
25121 0     ALP3946         1.0         2.430763        8   
25122 0     ALP3946         1.0       116.325330        8   
25123 0     ALP3946         1.0        66.036976        8   

                            information_link  shape_Length    shape_Area  \
0     0               https://www.osmre.gov/   1220.769913  1.354382e+04   
1     0               https://www.osmre.gov/    860.404674  1.678249e+04   
2     0               https://www.osmre.gov/   7219.650616  1.689153e+06   
3     0               https://www.osmre.gov/    688.042643  2.188255e+04   
      1               https://www.osmre.gov/    688.042643  2.188255e+04   
...                                      ...           ...           ...   
25119 0  https://surface-mining.alabama.gov/   5370.984081  9.330482e+05   
25120 0  https://surface-mining.alabama.gov/    534.608228  1.452293e+04   
25121 0  https://surface-mining.alabama.gov/    507.253927  1.464629e+04   
25122 0  https://surface-mining.alabama.gov/   4223.617167  7.008642e+05   
25123 0  https://surface-mining.alabama.gov/   3278.442601  3.979045e+05   

                                                  geometry  
0     0  POLYGON ((-9515951.177 4212750.257, -9515928.8...  
1     0  POLYGON ((-9516024.476 4212708.540, -9516019.2...  
2     0  POLYGON ((-9459511.615 4347543.957, -9459509.4...  
3     0  POLYGON ((-9372128.011 4337061.595, -9372159.8...  
      1  POLYGON ((-9372169.449 4337098.162, -9372176.4...  
...                                                    ...  
25119 0  POLYGON ((-9540804.196 4143151.210, -9540794.6...  
25120 0  POLYGON ((-9541424.447 4142622.814, -9541448.6...  
25121 0  POLYGON ((-9540236.664 4142842.258, -9540258.9...  
25122 0  POLYGON ((-9540725.233 4143212.597, -9540601.6...  
25123 0  POLYGON ((-9539594.620 4143493.033, -9539552.9...  

[54301 rows x 12 columns]

In [ ]:
i=0

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

df_contructor = []
while i<=len(mines_exploded):
 try:
        # Get dynamic world image collection by asset ID
        dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date)

        # get band names, equal to column names
        bandnames = dc.first().bandNames().getInfo()[:-1]

        polygon = mines_exploded['geometry'].values[i]

        # Extract the coordinates from the polygon

        #needs adjusting for different geometry types
        coordinates = polygon.exterior.coords[:]

        # Create an ee.Geometry object from the coordinates
        polygon = ee.Geometry.MultiPolygon(coordinates)

        # Set the maxError parameter
        max_error = 1  # Specify your desired maxError value

        # Project the geometry coordinates
        projected_polygon = polygon.transform('EPSG:4326', max_error)

        #decrease scale to get more rows of data/granularity
        ac = dc.getRegion(projected_polygon, scale=100 ).getInfo()
        df_constructor =pd.DataFrame(ac)

        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['company'] = mines_exploded['company'].values[i]
        df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
        df_constructor['start_date'] = i_date
        df_constructor['end_Date'] = f_date
        df = df.append(df_constructor)

        print(mines_exploded['company'].values[i])
        i+=1
 except:
          i+=1
          pass



In [ ]:
df['company'].unique()

array([nan, 'COALCO INC', 'ENCOL INC', 'DEBCO COAL CO',
       'B + J EXCAVATING CO INC (TN)', 'CUMBERLAND COAL CO LLC',
       'PREMIUM COAL CO. INC', 'ROBINSON COAL CO',
       'SEQUATCHIE VALLEY COAL CORP', 'WHITE OAK COAL CORP',
       'COAL ENERGY INC', 'D + H COAL CO INC',
       'META ELKHORN MINING CO INC', 'ESTATE OF BRUNO GERNT INC',
       'KOPPER GLO FUEL INC', 'COLQUEST ENERGY INC',
       'EAST TENNESSEE RESOURCES INC', 'PLYMOUTH COAL CO',
       'S + H MINING INC', 'HAMBLIN COAL CO',
       'OLIVER SPRINGS MINING CO INC', 'TENNESSEE CONSOLIDATED COAL C0',
       'MARION COAL CO INC', 'PATTON MINING CO',
       'ADDINGTON ENTERPRISES INC', 'LCC TENNESSEE  LLC',
       'REX MINING INC', 'TRIPLE H COAL  LLC.', 'TENNESSEE MINING INC',
       'STONEY RIDGE COAL COMPANY INC', 'S R S COAL INC',
       'NATIONAL COAL CORPORATION', 'U S COAL INC', 'SHEMCO INC',
       'VOLUNTEER PROCESSING INC', 'MEADOWS COAL CO',
       'INDUSTRIAL PROCESSING COMPANY', 'ROBERT CLEAR COAL CORP',


In [ ]:
df

,id,longitude,latitude,time,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice,label,company,shapeName
1,20210104T162701_20210104T162655_T16SGF,-84.313178,36.291039,1609778000459,0.04542,0.274786,0.092658,0.039822,0.038571,0.303898,0.032706,0.121162,0.034524,1,NaN,NaN
2,20210104T162701_20210104T162655_T17SKA,-84.313178,36.291039,1609777999068,0.042026,0.290325,0.087177,0.038763,0.04232,0.29151,0.03383,0.12493,0.034269,5,NaN,NaN
3,20210129T162529_20210129T163010_T16SGF,-84.313178,36.291039,1611937999778,0.046217,0.258137,0.105504,0.026195,0.042333,0.265838,0.038878,0.15122,0.041957,5,NaN,NaN
4,20210129T162529_20210129T163010_T17SKA,-84.313178,36.291039,1611937998386,0.042931,0.266436,0.109466,0.02528,0.046427,0.248172,0.035881,0.158534,0.039877,5,NaN,NaN
1,20210104T162701_20210104T162655_T16SGF,-84.539553,36.298226,1609778000459,0.050889,0.388929,0.057669,0.038968,0.028941,0.279445,0.027273,0.086808,0.031611,1,COALCO INC,0000000000000000000f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,20210104T162701_20210104T162655_T16SGF,-84.345068,36.2852,1609778000459,0.069676,0.498362,0.074818,0.040029,0.038362,0.145228,0.039846,0.053292,0.03785,1,U S COAL INC,00000000000000000003
30,20210104T162701_20210104T162655_T16SGF,-84.344169,36.2852,1609778000459,0.069764,0.354117,0.079024,0.039367,0.043649,0.254445,0.040096,0.069402,0.044334,1,U S COAL INC,00000000000000000003
31,20210104T162701_20210104T162655_T16SGF,-84.345966,36.286098,1609778000459,0.055175,0.222323,0.069861,0.037601,0.052108,0.207541,0.224025,0.066515,0.042887,6,U S COAL INC,00000000000000000003
32,20210104T162701_20210104T162655_T16SGF,-84.345068,36.286098,1609778000459,0.069354,0.358745,0.090561,0.045413,0.046053,0.223375,0.054247,0.065136,0.039227,1,U S COAL INC,00000000000000000003


In [ ]:
len(df['company'].unique())

48

In [ ]:
len(mines_exploded['company'].unique())

27